In [1]:
import tensorflow.compat.v2 as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pathlib
import random
import glob
import os
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import time
keras = tf.keras
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32)
    image = image/255.0
    image = 2*image-1
    return image

In [3]:
train_dir = r'C:\Users\24306\Desktop\datasets\kfold\4\train'
test_dir  = r'C:\Users\24306\Desktop\datasets\kfold\4\test'
checkpoint_path = r'C:\Users\24306\Desktop\NASNetLarge\4\NASNetLarge.ckpt'

# 处理训练数据

In [4]:
#打乱顺序
train_root = pathlib.Path(train_dir)
train_image_paths = list(train_root.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)
#处理训练标签
train_image_labels = [pathlib.Path(path).parent.name for path in train_image_paths]
train_labels = [[label.split('_')[0]] for label in train_image_labels]
new_train_labels=[]
for n in train_labels:
    new_train_labels.append(float(n[0]));  
train_labels=new_train_labels
print(train_labels)
#处理训练图像
train_path_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_image_ds = train_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
train_label_ds = tf.data.Dataset.from_tensor_slices(train_labels)
train_image_label_ds = tf.data.Dataset.zip((train_image_ds, train_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
train_count = len(train_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
train_data = train_image_label_ds.shuffle(buffer_size=train_count).repeat(-1)
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(train_data)



[32.0, 16.0, 16.0, 52.0, 37.0, 66.0, 96.0, 47.0, 37.0, 55.0, 32.0, 7.0, 13.0, 16.0, 45.0, 100.0, 37.0, 47.0, 93.0, 100.0, 45.0, 37.0, 47.0, 45.0, 40.0, 45.0, 45.0, 84.0, 40.0, 47.0, 93.0, 23.0, 16.0, 93.0, 100.0, 80.0, 13.0, 37.0, 13.0, 45.0, 95.0, 13.0, 47.0, 7.0, 95.0, 84.0, 80.0, 40.0, 100.0, 40.0, 100.0, 10.0, 100.0, 93.0, 80.0, 37.0, 23.0, 16.0, 55.0, 40.0, 23.0, 100.0, 95.0, 39.0, 16.0, 84.0, 86.0, 37.0, 80.0, 100.0, 52.0, 47.0, 47.0, 45.0, 16.0, 84.0, 84.0, 95.0, 30.0, 96.0, 7.0, 95.0, 16.0, 47.0, 35.0, 100.0, 93.0, 7.0, 30.0, 100.0, 37.0, 10.0, 84.0, 80.0, 4.0, 45.0, 16.0, 93.0, 100.0, 96.0, 13.0, 84.0, 96.0, 20.0, 30.0, 95.0, 84.0, 16.0, 96.0, 80.0, 40.0, 100.0, 45.0, 16.0, 80.0, 16.0, 95.0, 13.0, 47.0, 7.0, 84.0, 80.0, 16.0, 13.0, 30.0, 47.0, 80.0, 84.0, 16.0, 10.0, 80.0, 86.0, 47.0, 37.0, 55.0, 37.0, 16.0, 13.0, 96.0, 20.0, 47.0, 80.0, 84.0, 100.0, 37.0, 40.0, 84.0, 66.0, 37.0, 95.0, 10.0, 40.0, 37.0, 4.0, 52.0, 95.0, 37.0, 84.0, 45.0, 66.0, 84.0, 45.0, 37.0, 52.0, 45.0, 66.

# 处理验证数据

In [5]:
#打乱顺序
test_root = pathlib.Path(test_dir)
test_image_paths = list(test_root.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths]
#random.shuffle(train_image_paths)
#处理训练标签
test_image_labels = [pathlib.Path(path).parent.name for path in test_image_paths]
test_labels = [[label.split('_')[0]] for label in test_image_labels]
new_test_labels=[]
for n in test_labels:
    new_test_labels.append(float(n[0]));  
test_labels=new_test_labels
print(test_labels)
#处理训练图像
test_path_ds = tf.data.Dataset.from_tensor_slices(test_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
test_image_ds = test_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
test_label_ds = tf.data.Dataset.from_tensor_slices(test_labels)
test_image_label_ds = tf.data.Dataset.zip((test_image_ds, test_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
test_count = len(test_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
test_data = test_image_label_ds.shuffle(buffer_size=test_count).repeat(-1)
test_data = test_data.batch(BATCH_SIZE)
test_data = test_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(test_data)

[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.

In [6]:
#要改
NASNetLarge = tf.keras.applications.nasnet.NASNetLarge(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
#要改
NASNetLarge.trainable = True
inputs = tf.keras.Input(shape=(224, 224, 3))
#要改
x = NASNetLarge(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
out_weight = tf.keras.layers.Dense(1,name='out_weight')(x)

model = tf.keras.Model(inputs=inputs,
                       outputs=[out_weight])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 NASNet (Functional)         (None, 7, 7, 4032)        84916818  
                                                                 
 global_average_pooling2d (G  (None, 4032)             0         
 lobalAveragePooling2D)                                          
                                                                 
 out_weight (Dense)          (None, 1)                 4033      
                                                                 
Total params: 84,920,851
Trainable params: 84,724,183
Non-trainable params: 196,668
_________________________________________________________________


In [7]:
print ('start time:',time.asctime( time.localtime(time.time()) ))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.01),loss='mse')

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_best_only=True,monitor='val_loss')
history = model.fit(train_data,
                    steps_per_epoch=train_count//BATCH_SIZE,
                    epochs=300,
                    validation_data=test_data,
                    validation_steps=test_count//BATCH_SIZE,
                   callbacks=[cp_callback])
#要改
np.savetxt('NASNetLarge_loss_4.txt',history.history['loss'])
#要改
np.savetxt('NASNetLarge_valloss_4.txt',history.history['val_loss'])
print ('end time:',time.asctime( time.localtime(time.time()) ))

start time: Mon Apr 24 16:11:16 2023
Epoch 1/300
592/592 [==============================] - ETA: 0s - loss: 311.6935

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetLarge\4\NASNetLarge.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetLarge\4\NASNetLarge.ckpt\assets


592/592 [==============================] - 405s 609ms/step - loss: 311.6935 - val_loss: 3313.9331
Epoch 2/300
592/592 [==============================] - ETA: 0s - loss: 4.8419

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetLarge\4\NASNetLarge.ckpt\assets


...

592/592 [==============================] - 393s 664ms/step - loss: 0.0123 - val_loss: 0.0250
Epoch 36/300
592/592 [==============================] - 263s 444ms/step - loss: 0.0129 - val_loss: 0.2940
Epoch 37/300
592/592 [==============================] - 263s 444ms/step - loss: 0.0123 - val_loss: 0.1842
Epoch 38/300
592/592 [==============================] - ETA: 0s - loss: 0.0970

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetLarge\4\NASNetLarge.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetLarge\4\NASNetLarge.ckpt\assets


592/592 [==============================] - 395s 667ms/step - loss: 0.0970 - val_loss: 0.0072
Epoch 39/300
592/592 [==============================] - 264s 445ms/step - loss: 0.0132 - val_loss: 0.2164
Epoch 40/300
592/592 [==============================] - 267s 451ms/step - loss: 0.0112 - val_loss: 0.0776
Epoch 41/300
592/592 [==============================] - 275s 464ms/step - loss: 0.0118 - val_loss: 0.0358
Epoch 42/300
592/592 [==============================] - 276s 466ms/step - loss: 0.0075 - val_loss: 0.0598
Epoch 43/300
592/592 [==============================] - 276s 465ms/step - loss: 0.0163 - val_loss: 0.0080
Epoch 44/300
592/592 [==============================] - 276s 466ms/step - loss: 0.0072 - val_loss: 0.1372
Epoch 45/300
592/592 [==============================] - 275s 464ms/step - loss: 0.0079 - val_loss: 0.2269
Epoch 46/300
592/592 [==============================] - 276s 466ms/step - loss: 0.0092 - val_loss: 0.0955
Epoch 47/300
592/592 [==============================] - 276

592/592 [==============================] - 278s 470ms/step - loss: 0.0093 - val_loss: 0.0204
Epoch 191/300
592/592 [==============================] - 278s 470ms/step - loss: 0.0013 - val_loss: 0.0526
Epoch 192/300
592/592 [==============================] - 277s 467ms/step - loss: 7.3317e-04 - val_loss: 0.0553
Epoch 193/300
592/592 [==============================] - 277s 468ms/step - loss: 7.4395e-04 - val_loss: 0.0521
Epoch 194/300
592/592 [==============================] - 278s 469ms/step - loss: 6.0313e-04 - val_loss: 0.0390
Epoch 195/300
592/592 [==============================] - 277s 468ms/step - loss: 7.1178e-04 - val_loss: 0.0471
Epoch 196/300
592/592 [==============================] - 277s 468ms/step - loss: 0.0039 - val_loss: 0.0627
Epoch 197/300
592/592 [==============================] - 278s 469ms/step - loss: 8.6898e-04 - val_loss: 0.0570
Epoch 198/300
592/592 [==============================] - 278s 469ms/step - loss: 6.2787e-04 - val_loss: 0.0733
Epoch 199/300
592/592 [====